In [ ]:
import json
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib

# Individual Strings Results

In [ ]:
method = 'gcg'
logdir = f'results/'

# for individual experiments
individual = True
mode = 'behaviors'

files = !ls {logdir}individual_{mode}_*_ascii*
files = [f for f in files if 'json' in f]
files = sorted(files, key=lambda x: "_".join(x.split('_')[:-1]))

max_examples = 100

logs = []
for logfile in files:
    with open(logfile, 'r') as f:
        logs.append(json.load(f))
log = logs[0]
len(logs)

In [ ]:
config = log['params']
print(config.keys())

total_steps = config['n_steps']
test_steps = config.get('test_steps', 50)
log_steps = total_steps // test_steps + 1
print('log_steps', log_steps)

if individual:
    examples = 0
    test_logs = []
    control_logs = []
    goals, targets = [],[]
    for l in logs:
        sub_test_logs = l['tests']
        sub_examples = len(sub_test_logs) // log_steps
        examples += sub_examples
        test_logs.extend(sub_test_logs[:sub_examples * log_steps])
        control_logs.extend(l['controls'][:sub_examples * log_steps])
        goals.extend(l['params']['goals'][:sub_examples])
        targets.extend(l['params']['targets'][:sub_examples])
        if examples >= max_examples:
            break
else:
    test_logs = log['tests']
    examples = 1

passed, em, loss, total = [],[],[],[]
for i in range(examples):
    sub_passed, sub_em, sub_loss, sub_total = [],[],[],[]
    for res in test_logs[i*log_steps:(i+1)*log_steps]:
        sub_passed.append(res['n_passed'])
        sub_em.append(res['n_em'])
        sub_loss.append(res['n_loss'])
        sub_total.append(res['total'])
    passed.append(sub_passed)
    em.append(sub_em)
    loss.append(sub_loss)
    total.append(sub_total)
passed = np.array(passed)
em = np.array(em)
loss = np.array(loss)
total = np.array(total)
total.shape

In [ ]:
em[...,0].mean(0)[-1], loss[...,0].mean(0)[-1]

# Individual Behaviors Results

(To get more accurate results, please run the cells below, then use `evaluate_individual.py`)

In [26]:
# method = 'gcg'
# logdir = f'results/'

# # for individual experiments
# individual = True
# mode = 'behaviors'
# model = "llama2"

# files = !ls {logdir}individual_{model}_{mode}
# print(files)
# files = [f for f in files if 'json' in f]
# files = sorted(files, key=lambda x: "_".join(x.split('_')[:-1]))

# max_examples = 100
# print(files)

# logs = []
# for logfile in files:
#     with open(logfile, 'r') as f:
#         logs.append(json.load(f))
# log = logs[0]
# len(logs)

["ls: cannot access 'results/individual_llama2_behaviors': No such file or directory"]
[]


IndexError: list index out of range

In [7]:
import os
import re
import json
individual = True
root = "/home/liao.629/llm-attacks/experiments/results"
pattern = re.compile(r"individual_behaviors_llama2_gcg.*json")

x = [path for path in os.listdir(root) if pattern.match(path)]

logs = []
for logfile in x:
    with open(os.path.join(root,logfile), 'r') as f:
        logs.append(json.load(f))
log = logs[0]

In [8]:
len(logs)

52

In [10]:
config = log['params']
print(config.keys())

total_steps = config['n_steps']
test_steps = config.get('test_steps', 50)
log_steps = total_steps // test_steps + 1
print('log_steps', log_steps)

if individual:
    examples = 0
    test_logs = []
    control_logs = []
    goals, targets = [],[]
    for l in logs:
        sub_test_logs = l['tests']
        sub_examples = len(sub_test_logs) // log_steps
        examples += sub_examples
        test_logs.extend(sub_test_logs[:sub_examples * log_steps])
        control_logs.extend(l['controls'][:sub_examples * log_steps])
        goals.extend(l['params']['goals'][:sub_examples])
        targets.extend(l['params']['targets'][:sub_examples])
        # if examples >= max_examples:
        #     break
else:
    test_logs = log['tests']
    examples = 1
examples

dict_keys(['goals', 'targets', 'test_goals', 'test_targets', 'control_init', 'test_prefixes', 'models', 'test_models', 'n_steps', 'test_steps', 'batch_size', 'topk', 'temp', 'allow_non_ascii', 'target_weight', 'control_weight', 'anneal', 'incr_control', 'stop_on_success'])
log_steps 3


520

In [11]:
sub_test_logs = logs[0]["tests"]
len(sub_test_logs) // log_steps

10

In [12]:
log_steps

3

In [13]:
len(control_logs)

1560

In [14]:
import numpy as np
passed, em, loss, total, controls = [],[],[],[],[]
for i in range(examples):
    sub_passed, sub_em, sub_loss, sub_total, sub_control = [],[],[],[],[]
    for res in test_logs[i*log_steps:(i+1)*log_steps]:
        sub_passed.append(res['n_passed'])
        sub_em.append(res['n_em'])
        sub_loss.append(res['n_loss'])
        sub_total.append(res['total'])
    sub_control = control_logs[i*log_steps:(i+1)*log_steps]
    passed.append(sub_passed)
    em.append(sub_em)
    loss.append(sub_loss)
    total.append(sub_total)
    controls.append(sub_control)
passed = np.array(passed)
em = np.array(em)
loss = np.array(loss)
total = np.array(total)
total.shape

(520, 3, 4)

In [15]:
saved_controls

NameError: name 'saved_controls' is not defined

In [16]:
loss.shape

(520, 3, 4)

In [17]:
saved_controls = [c[-1] for c in controls]
json_obj = {
    'goal': goals,
    'target': targets,
    'controls': saved_controls
}
with open('results/individual_behavior_controls.json', 'w') as f:
    json.dump(json_obj, f)

# now run `evaluate_individual.py` with this file

In [40]:
len(saved_controls)

100

In [37]:
data = json.load(open('eval/individual_behavior_controls.json', 'r'))

FileNotFoundError: [Errno 2] No such file or directory: 'eval/individual_behavior_controls.json'

In [ ]:
(np.array(data['Vicuna-7B']['jb']) == 1).mean()

# Transfer Results

Run `evaluate.py` on the logfile first to generate a log in the eval dir

In [ ]:
def plot_log(log, jb, idx=-1):
    fig, axes = plt.subplots(1, 3, figsize=(15, 3))

    # Plotting the bars in the first plot
    bars = axes[0].bar(log.keys(), jb[:, idx])
    axes[0].xaxis.set_tick_params(rotation=90)
    axes[0].grid(axis='y', ls='dashed')

    # Plotting the lines in the second plot
    lines = []
    for i in range(len(log)):
        line, = axes[1].plot(range(len(jb[0])), jb[i], label=list(log.keys())[i])
        lines.append(line)

    # Getting the handles and labels from the legend of the second plot
    handles, labels = axes[1].get_legend_handles_labels()

    # Plotting the legend in the first plot using the handles and labels from the second plot
    axes[0].legend(handles=handles, labels=labels, bbox_to_anchor=(1.1, -0.45, 2., .102),
                loc='lower left', ncol=4, mode="expand", borderaxespad=0.)

    axes[2].plot(range(len(jb[0])), jb.mean(0), color='red')
    axes[2].set_ylim(0, 100)
    axes[2].grid(axis='y', ls='dashed')

    # Matching the colors of the bars in the first plot with the lines in the legend
    for bar, line in zip(bars, lines):
        bar.set_color(line.get_color())

    plt.show()

In [ ]:
logdir = f'eval/'
logfile = <your_logfile>

with open(logdir + logfile, 'r') as f:
    log = json.load(f)

In [ ]:
jb, em = [],[]
for model in log:
    stats = log[model]
    jb.append(stats['test_jb'])
    em.append(stats['test_em'])
jb = np.array(jb)
jb = jb.mean(-1)
em = np.array(em)
em = em.mean(-1)

In [ ]:
plot_log(log, jb, idx=-1)